In [ ]:
import numpy as np
import pandas as pd
import m3utils
from sklearn.ensemble import RandomForestRegressor
import plotly.express as px
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True) # falls plotly mal nichts anzeigt, diese Zelle wieder ausführen

In [ ]:
!pip install pmdarima

In [ ]:
df = m3utils.load_covid_19_data()

In [ ]:
def train_predict(features):
    df = features.copy()
    df['Country/Region'] = df['Country/Region'].cat.codes
    df['date'] = m3utils.date_to_days(df['date'])
    
    result = m3utils.train_predict(df, 7, 'Country/Region', 'date', 'value_daily', 'value_daily_predicted', RandomForestRegressor(), ['date'])
    
    result['date'] = m3utils.days_to_date(result['date'])
    result['Country/Region'] = features['Country/Region'].cat.categories[result['Country/Region'].astype('int')]
    return result

m3utils.crossvalidate(df, 'Country/Region', 'date', 'value_daily', 'value_daily_predicted', train_predict, pd.to_datetime('2020-09-01'), 5, 28, m3utils.wmape)

# Kombination mit klassischen Zeitreihen-Verfahren
Dieses Notebook zeigt, wie klassische Zeitreihen-Verfahren in Kombination mit den üblichen Machine-Learning-Algorithmen genutzt werden können. Die klassischen Zeitreihen-Verfahren haben einige Vorteile.

<img src="https://i.ibb.co/bPnPnT8/tsalgos.jpg" border="0"/>

- Trend und Saisonalität: sind direkt eingebaute Konzepte
- Datensparsamkeit: pressen aus wenigen Daten alles raus, z.B. reichen 2 Jahre Daten, um die jährliche Saisonalität zu bestimmen
- Flexibilität: wenn z.B. ein neues Produkt kommt und dafür noch keine Saisonalität bekannt ist, können wir auf die Saisonalität vom Gesamtmarkt oder einem ähnlichen Produkt ausweichen
- Extrapolation: anders als die baumbasierten-Verfahren sind die Zeitreihen-Verfahren sehr gut darin zu extrapolieren – also einen Trend in die Zukunft fortzusetzen.

Die Pakete [statsmodels](https://www.statsmodels.org) und [pmdarima](http://alkaline-ml.com/pmdarima/) enthalten viele gängige Zeitreihen-Verfahren.

## Dekomposition der Zeitreihe in Trend, Saisonalität und Residuum

Die Methode `seasonal_decompose` nimmt die Dekomposition einer Zeitreihe in Trend, Saisonalität und Residuum vor. Das Standardmodell ist additiv: $Y_t=T_t+S_t+R_t$

Für die Dekomposition wird zuerst die Trend-Komponente bestimmt. Im Standardfall passiert dies mit einem gleitenden Durchschnitt über ein Fenster der Größe der Periode der Zeitreihe. Die Periode der Zeitreihe wird aus der zeitlichen Auflösung der Zeitreihe und dem saisonalen Muster bestimmt:

* **monatliche Daten, jährliche Saisonalität** (Monate sind jedes Jahr ähnlich): `period=12`
* **tägliche Daten, wöchentliche Saisonalität** (Tage sind jede Woche ähnlich): `period=7`
* **quartalsweise Daten, jährliche Saisonalität** (Quartale sind jedes Jahr ähnlich): `period=4`
* **stündliche Daten, tägliche Saisonalität** (Stunden sind jeden Tag ähnlich): `period=24`
* **tägliche Daten, jährliche Saisonalität** (Tage sind jedes Jahr ähnlich): `period=?` (Schaltjahre sind doof)
* **wöchentliche Daten, jährliche Saisonalität** (Wochen sind jedes Jahr ähnlich): `period=?` (Arithmetik mit Kalenderwochen ist auch doof)

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
df2 = df.set_index('date')
s = df2.loc[df2['Country/Region']=='Germany', 'value_daily']

decomp = seasonal_decompose(s, period=7)
plot_df = pd.DataFrame({'date': s.index, 'original': s, 'trend': decomp.trend, 'seasonal': decomp.seasonal, 'residuals': decomp.resid}).melt(id_vars='date')

fig = px.line(plot_df, x='date', y='value', facet_row='variable')
fig.update_yaxes(matches=None)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
fig.show()

Das wöchentliche Muster in der Zeitreihe ist gut in der Saisonalitätskomponente zu erkennen. Problematisch ist, dass das Residuum ein klares Muster bei niedrigen Levels der Zeitreihe aufweist und die Varianz des Residuums bei hohen Levels steigt. Das additive Modell, das wir für die Dekomposition genutzt haben passt in diesem Fall nicht und wahrscheinlich ist ein multiplikativer Ansatz besser: $Y_t=T_t \cdot S_t \cdot R_t$ 

In [ ]:
decomp = seasonal_decompose(s+1, model='m', period=7) # im Multiplikativen Modell sind keine Werte=0 erlaubt
plot_df = pd.DataFrame({'date': s.index, 'original': s, 'trend': decomp.trend, 'seasonal': decomp.seasonal, 'residuals': decomp.resid}).melt(id_vars='date')

fig = px.line(plot_df, x='date', y='value', facet_row='variable')
fig.update_yaxes(matches=None)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[1]))
fig.show()

# Slippery when wet: Data Leakage

Die statistischen Verfahren operieren auf der gesamten Zeitreihe. Dadurch kann es dazu kommen, dass wir beim Feature Engineering Informationen aus der Zukunft nutzen, was wir ja dringend vermeiden möchten. Ein Beispiel dafür...

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

ts = df.loc[df['Country/Region']=='Germany', 'value_daily']
trend = seasonal_decompose(ts, model='additive', two_sided=True, period=7).trend

ts.loc[1797:] = 0

trend_with_break = seasonal_decompose(ts, model='additive', two_sided=True, period=7).trend # ändern auf two_sided=False

fig = px.line(pd.DataFrame({'original': trend, 'with break': trend_with_break}))

fig.add_vline(x=1797, line_dash="dash")

# Beispiel für eine Vorhersage mit ARIMA

Für die Vorhersagen von Zeitreihen können wir z.B. ein ARIMA-Modell benutzen. Die Klasse [`AutoARIMA`](https://alkaline-ml.com/pmdarima/modules/generated/pmdarima.arima.AutoARIMA.html) aus pmdarima ermittelt automatisch die am besten passenden Hyperparameter für uns.


In [ ]:
%%time
import pmdarima as pm

ts = df.loc[df['Country/Region']=='Germany', 'value_daily']
# Laufzeit ca. 1-2 Minute
#arima = pm.auto_arima(ts, error_action='ignore', suppress_warnings=True, maxiter=10, seasonal=True, m=7)

Weil die Methode aber eine Weile braucht, können wir die passenden Parameter auch selber an die Klasse [ARIMA](https://alkaline-ml.com/pmdarima/modules/generated/pmdarima.arima.ARIMA.html) übergeben und mit der Methode `fit` das Training auslösen.

In [ ]:
arima = pm.arima.ARIMA((5,1,5),(2,0,2,7), with_intercept=False, suppress_warnings=True)
arima.fit(ts)

Die Methode `predict` auf dem trainierten Objekt macht dann die Vorhersage

In [ ]:
prediction = arima.predict(7)
prediction

In [ ]:
forecast_date = df['date'].max()

result = pd.DataFrame()
result['date'] = m3utils.days_to_date(pd.Series(range(1,7+1)), forecast_date)
result['Country/Region'] = 'Germany'
result['Country/Region'] = result['Country/Region'].astype(df['Country/Region'].dtype)
result['value_daily_predicted'] = prediction
result

result = result.append(df.loc[(df['date'] <= forecast_date) & (df['Country/Region']=='Germany')][['Country/Region', 'date', 'value_daily']])
result

px.line(result,
        x='date', y=['value_daily_predicted', 'value_daily'],
        title='Vorhersage mit ARIMA-Modell', labels=dict(date='Datum', value=''))

## Weiterführende Informationen zu den Zeitreihen-Algos

https://otexts.com/fpp3/

<img src="https://otexts.com/fpp3/figs/fpp2_cover.jpg"/>

# Übung

## 1. Aufgabe - Vorhersage mit ARIMA

Wir starten damit, das Du eine Validierung der Vorhersagen mit ARIMA machst. Schreibe dafür eine Methode `train_predict_arima`, die die Daten annimmt und dann für jedes Land (`Country/Region`) ein ARIMA-Modell fitted und die Vorhersage macht. Du kannst die Hyperparameter für das ARIMA-Modell von oben übernehmen. Die Methode `train_predict_arima` sollte einen DataFrame zurückliefern mit den Spalten `date`, `Country/Region` und `value_daily_predicted`. **Eine gute Vorlage dafür ist der Code von oben, der für den Plot der Vorhersage genutzt wurde.**

Erstelle zuerst den Code für die Vorhersage eines Landes, dann für mehrere Länder und verpacke erst dann den Code die Methode und mache eine Crossvalidation.

In [ ]:
# Hier ausprobieren

In [ ]:
def train_predict_arima(features):
    # Dein Code hier
    return result

In [ ]:
# Hier validieren
#m3utils.crossvalidate(df, 'Country/Region', 'date', 'value_daily', 'value_daily_predicted', train_predict_arima, pd.to_datetime('2020-09-01'), 5, 28, m3utils.wmape)

## 2. Aufgabe - Saisonalität außerhalb vom ML-Modell [optional]

Um die Vorhersagen von ARIMA mit einem ML-Modell zu kombinieren, brauchen wir Ensemble-Techniken, die wir noch nicht vorgestellt haben. Es gibt aber eine andere Möglichkeit von den statistischen Verfahren zu profitieren. Du kannst die Saisonalität, die wir oben mit der Methode `seasonal_decompose` aus den Daten entfernen bevor Du das ML-Modell trainierst und die Saisonalität dann wieder den Vorhersagen hinzufügen.

In [ ]:
# hier ausprobieren

In [ ]:
def train_predict_decompose(features):
    df = features.copy()
    df['Country/Region'] = df['Country/Region'].cat.codes
    df['date'] = m3utils.date_to_days(df['date'])
    
    df['seasonal'] = 1 # hier statt 1 die Saisonalität je Gruppe mit Pandas apply und seasonal_decompose ermitteln
    # hier die Saisonalität aus value_daily entfernen
    
    result = m3utils.train_predict(df, 7, 'Country/Region', 'date', 'value_daily', 'value_daily_predicted', RandomForestRegressor(), ['date', 'seasonal'])
    
    # hier die Saisonalität je Gruppe wieder hinzufügen, Idee: die letzten 7 aus df mit Pandas GroupBy.tail(7) ermitteln
    
    result['date'] = m3utils.days_to_date(result['date'])
    result['Country/Region'] = features['Country/Region'].cat.categories[result['Country/Region'].astype('int')]
    return result

In [ ]:
# Hier validieren
#m3utils.crossvalidate(df, 'Country/Region', 'date', 'value_daily', 'value_daily_predicted', train_predict_arima, pd.to_datetime('2020-09-01'), 5, 28, m3utils.wmape)

## Musterlösung: 1. Aufgabe - Vorhersage mit ARIMA

In [ ]:
def train_predict_arima(df):
    forecast_date = df['date'].max()
    arima = pm.arima.ARIMA((5,1,5),(2,0,2,7), with_intercept=False, suppress_warnings=True)

    result = pd.DataFrame()

    countries = df['Country/Region'].unique()
    for country in countries:
        arima.fit(df.loc[df['Country/Region']==country]['value_daily'])
        prediction = arima.predict(7)

        tmp = pd.DataFrame()
        tmp['date'] = m3utils.days_to_date(pd.Series(range(1,7+1)), forecast_date)
        tmp['Country/Region'] = country
        tmp['Country/Region'] = tmp['Country/Region'].astype(df['Country/Region'].dtype)
        tmp['value_daily_predicted'] = prediction
        result = result.append(tmp)
    return(result)

In [ ]:
m3utils.crossvalidate(df, 'Country/Region', 'date', 'value_daily', 'value_daily_predicted', train_predict_arima, pd.to_datetime('2020-09-01'), 5, 28, m3utils.wmape)

## Musterlösung: 2. Aufgabe - Saisonalität außerhalb vom ML-Modell [optional]

In [ ]:
def train_predict_decompose(features):
    df = features.copy()
    df['Country/Region'] = df['Country/Region'].cat.codes
    df['date'] = m3utils.date_to_days(df['date'])
    
    df['seasonal'] = df.groupby('Country/Region', observed=True)['value_daily'].apply(lambda x: seasonal_decompose(x+1, model='m', period=7).seasonal)
    df['value_daily'] = df['value_daily']/df['seasonal']
    
    result = m3utils.train_predict(df, 7, 'Country/Region', 'date', 'value_daily', 'value_daily_predicted', RandomForestRegressor(), ['date'])
    
    seasonal = df[['Country/Region', 'date', 'seasonal']].groupby('Country/Region', observed=True).tail(7).copy()
    seasonal['date'] = seasonal['date'] + 7
    result = result.merge(seasonal, on=['Country/Region', 'date'])
    result['value_daily_predicted_unadjusted'] = result['value_daily_predicted']
    result['value_daily_predicted'] = result['value_daily_predicted'] * result['seasonal']
    
    result['date'] = m3utils.days_to_date(result['date'])
    result['Country/Region'] = features['Country/Region'].cat.categories[result['Country/Region'].astype('int')]
    return result

In [ ]:
m3utils.crossvalidate(df, 'Country/Region', 'date', 'value_daily', 'value_daily_predicted', train_predict_decompose, pd.to_datetime('2020-09-01'), 5, 28, m3utils.wmape)